# Frauchiger-Renner thought experiment in the Copenhagen Interpretation

### Installation instruction

It is recommended that you clone the qthought repository to your local machine and then run

in the qthought folder.
If you did not pip install qthought, you can use the following quick-fix by uncommenting and adapting to your local file path

In [ ]:
#import sys
#import os
# to run the example, set the following path to the folder path of qthought on your machine
#sys.path.append(os.path.abspath('/Users/nuri/qthought/qthought'))

### Defining the protocol

The code below implements the Frauchiger-Renner thought experiment where agents use Copenhagen interpretation to derive their predictions. Here agents come to a contradiction (''winning condition'') with probability $\frac{1}{12}$. Prior to reading this, it is recommended to take a look at the PDF description file Frauchiger-Renner example.

First, we import the ProjectQ operations needed for the protocol: the required single-qubit gates and the control. We also import *Protocol* and *ProtocolStep* classes to be able to define steps of the protocol; *QuantumSystem* to operate quantum systems of different dimensionality; *Agent* class and all functions from the *copenhagen_theory* module; *consistency* class to be able to chain agents' statements. Additionally, we import *InitR* function which initializes a qubit in the state $\frac{1}{\sqrt{3}} |0> + \sqrt{\frac{2}{3}} |1>$.

In [1]:
import sys
import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np

from projectq.ops import H, X, Measure, BasicGate
from projectq.meta import Control

from qthought.protocol import ProtocolStep
from qthought.quantumsystem import QuantumSystem
from qthought.agents import Agent
from qthought.interpretations.copenhagen_theory import *
from qthought.FrauchigerRennerExample.FR_protocol import InitR
from qthought.logicalReasoning.consistency import consistency

The first action of the protocol (at time $t=1$) is the initilization of the qubit $R$ Alice has in her lab in the state $\frac{1}{\sqrt{3}} |0> + \sqrt{\frac{2}{3}} |1>$. After defining the action, we define the step of the protocol by specifying: domain of action; written description of the action, which will be used for printouts during the run; time of the step; and which action variable being described.

In [2]:
# Step 1: Initialize r
# ----------------------------------------------------------
def step1_action(qsys):
    """Prepares the subsystem `r` of a `QuantumSystem` in the Frauchiger-Renner initial state."""
    InitR | qsys['r']


step1 = ProtocolStep(domain={'Qubit': ['r']},
                     descr='Initialize R',
                     time=1,
                     action=step1_action)

At $t=2$, Alice measures $R$ and writes the result in her memory.

In [3]:
# Step 2: Alice observes r
# ----------------------------------------------------------
def step2_action(qsys):
    observe(qsys['Alice_memory'], qsys['r'])


step2 = ProtocolStep(domain={'AgentMemory(1)': ['Alice'],
                             'Qubit': ['r']},
                      descr='ALICE observes R',
                      time=2,
                      action=step2_action)

At $t=3$, Alice makes an inference based on her outcome.

In [4]:
# Step 3: Alice makes inference
# ----------------------------------------------------------
def step3_action(qsys):
    qsys['Alice'].make_inference()


step3 = ProtocolStep(domain={'Agent(1,1)': ['Alice']},
                     descr='Alice makes an inference',
                     time=3,
                     action=step3_action)

At $t=4$, Alice prepares the qubit $S$ based on her outcome: in the state $|0>$ if she obtain $a=0$, and in the state $\frac{1}{\sqrt{2}} |0> + \frac{1}{\sqrt{2}} |1>$ if she got $a=1$. 

In [5]:
# Step 4: Alice prepares S
# ----------------------------------------------------------
def step4_action(qsys):
    with Control(qsys['eng'], qsys['Alice_memory']):
        H | qsys['s']


step4 = ProtocolStep(domain={'Qubit': ['s'],
                             'AgentMemory(1)': ['Alice']},
                     descr='Apply H to S controlled on ALICE_MEMORY',
                     time=4,
                     action=step4_action)

At $t=5$, Bob measures $S$ and writes the result down to his memory.

In [6]:
# Step 5: Bob measures S
# ----------------------------------------------------------
def step5_action(qsys):
    observe(qsys['Bob_memory'], qsys['s'])


step5 = ProtocolStep(domain={'Qubit': ['s'],
                             'AgentMemory(1)': ['Bob']},
                      descr='BOB measures S',
                      time=5,
                      action=step5_action)

At $t=6$, Bob makes an inference based on his outcome.

In [7]:
# Step 6: Bob makes inference
# ----------------------------------------------------------
def step6_action(qsys):
    qsys['Bob'].make_inference()


step6 = ProtocolStep(domain={'Agent(1,1)': ['Bob']},
                     descr='BOB makes an inference',
                     time=6,
                     action=step6_action)

At $t=7$, we need to reverse Alice's reasoning process for Ursula to be able to measure in the $|ok>$, $|fail>$ basis.

In [8]:
# Step 7: Reverse inference making in Alice
# ----------------------------------------------------------
def step7_action(qsys):
    qsys['Alice'].make_inference(reverse=True)
    observe(qsys['Alice_memory'], qsys['r'], reverse=True)


step7 = ProtocolStep(domain={'Agent(1,1)': ['Alice']},
                      descr='Reverse Alice reasoning (Step1: in ok --> 1(R)',
                      time=7,
                      action=step7_action)

Ursula measures Alice's lab in the $|ok>$, $|fail>$ basis (~ Bell basis). To do so, we first apply a Hadamard gate on $R$ at $t=8$, and then measure it in computational basis at $t=9$.

In [9]:
# Step 8: Hadamard on r
# ----------------------------------------------------------
def step8_action(qsys):
    H | qsys['r']


step8 = ProtocolStep(domain={'Qubit': ['r']},
                     descr='Perform Hadamard on R (Step2: in ok --> 1(R)',
                     time=8,
                     action=step8_action)

In [10]:
# Step 9: Ursula measures Alices lab
# ----------------------------------------------------------
def step9_action(qsys):
    observe(qsys['Ursula_memory'], qsys['r'])


step9 = ProtocolStep(domain={'Qubit': ['r'],
                             'AgentMemory(1)': ['Ursula']},
                      descr='URSULA measures ALICEs lab (i.e. r)',
                      time=9,
                      action=step9_action)

Ursula reasons based on her outcome at $t=10$, and announces it at $t=11$.

In [11]:
# Step 10: Ursula makes an inference
# ----------------------------------------------------------
def step10_action(qsys):
    qsys['Ursula'].make_inference()


step10 = ProtocolStep(domain={'Agent(1,1)': ['Ursula']},
                      descr='URSULA makes inference',
                      time=10,
                      action=step10_action)

In [12]:
# Step 11: Ursula announces her prediction
# ----------------------------------------------------------
def step11_action(qsys):
    Measure | qsys['Ursula_prediction']
    print('!Measurement made on Ursula_prediction!')
    print('Ursula prediction:', readout([qsys['Ursula_prediction']]))


step11 = ProtocolStep(domain={'Agent(1,1)': ['Ursula']},
                      descr='URSULA announces her prediction',
                      time=11,
                      action=step11_action)

Now we repeat the same procedure for Wigner measuring Bob's lab. First, we reverse Bob's reasoning process at $t=12$.

In [13]:
# Step 12: Reverse Bob's reasoning
# ----------------------------------------------------------
def step12_action(qsys):
    qsys['Bob'].make_inference(reverse=True)
    # qsys['Bob'].observe(qsys['s'], reverse=True)
    observe(qsys['Bob_memory'], qsys['s'], reverse=True)


step12 = ProtocolStep(domain={'Agent(1,1)': ['Bob']},
                       descr='Reverse BOBs inference procedure',
                       time=12,
                       action=step12_action)

Wigner measures Bob's lab in the $|ok>$, $|fail>$ basis (~ Bell basis). To do so, we first apply a Hadamard gate on $S$ at $t=13$, measure it in computational basis at $t=14$, and subsequently check if Wigner gets outcome "ok".

In [14]:
# Step 13: Apply Hadamard on s
# ----------------------------------------------------------
def step13_action(qsys):
    H | qsys['s']


step13 = ProtocolStep(domain={'Qubit': ['s']},
                      descr='Apply Hadamard on S, i.e. transform system S+BOB:  ok --> 1(s) ',
                      time=13,
                      action=step13_action)

In [15]:
# Step 14: Check if Bob is in ok state
# ----------------------------------------------------------
def step14_action(qsys):
    Measure | qsys['s']
    print('!Measurement made on s!')
    print('s-state:', readout([qsys['s']]))


step14 = ProtocolStep(domain={'Agent(1,1)': ['Bob']},
                      descr='Check if Bob+s is in ok state (corresponding to s: 1)',
                      time=14,
                      action=step14_action)

### Building up inference tables 

Now we construct the inference tables according to which the inference qubits of different agents are initialized. First, we consider the inference table of Alice: she has to reason about Wigner's outcome, and for that we need to include the steps of what is happening in the Bob's lab ($t=5,6$), and Wigner's actions ($t=12,13$).

In [16]:
p_TA_steps = [step1, step2, step3, step4, step5, step6,
         step12, step13]
p_TA = sum(p_TA_steps)
p_TA

Step 0: Initialize R(t:1)
Step 1: ALICE observes R(t:2)
Step 2: ALICE makes an inference(t:3)
Step 3: Apply H to S controlled on ALICE_MEMORY(t:4)
Step 4: BOB measures S(t:5)
Step 5: BOB makes an inference(t:6)
Step 6: Reverse BOBs inference procedure(t:12)
Step 7: Apply Hadamard on S, i.e. transform system S+BOB:  ok --> 1(s) (t:13)

Requirements: 
------------------------------
Qubit             ['r', 's']
Agent(1,1)        ['Bob', 'Alice']

Alice makes a forward inference about a measurement outcome later in the experiment, and get one determenistic conclusion $a=1 \Rightarrow w=fail$ ($w=fail$ here is equivalent to having measured $s=0$ at $t=13$).

In [17]:
TA = forward_inference(p_TA, 
                       subsys_x='Alice_memory', t_x=2, 
                       subsys_y='s', t_y=13, 
                       silent=True)
TA

(Note: This is the (slow) Python simulator.)


In:(Alice_memory:t2)  |  Out: (s:t13)
--------------------------------------------
           0          |        [0, 1]
           1          |        [0]

Now Bob reasons about Alice, making a backward inference about a measurement outcome earlier in the experiment.

In [18]:
p_TB_steps = [step1, step2, step4, step5]
p_TB = sum(p_TB_steps)
p_TB

Step 0: Initialize R(t:1)
Step 1: ALICE observes R(t:2)
Step 2: Apply H to S controlled on ALICE_MEMORY(t:4)
Step 3: BOB measures S(t:5)

Requirements: 
------------------------------
Qubit             ['r', 's']
AgentMemory(1)    ['Bob', 'Alice']

In [19]:
TB = backward_inference(p_TB, 
                        subsys_x='Alice_memory', t_x=2, 
                        subsys_y='Bob_memory', t_y=5, 
                        silent=False)
TB

(Note: This is the (slow) Python simulator.)
Require Qubit r
Require Qubit s
Require AgentMemory(1) Bob
Require AgentMemory(1) Alice
----- Case 0 -----
0 Initialize R t:1
State:
0.58|0000> + 0.82|0001>
1 ALICE observes R t:2
State:
0.58|0000> + 0.82|1001>
Projecting to subspace Alice_memory=0
1.0|0000>
2 Apply H to S controlled on ALICE_MEMORY t:4
State:
1.0|0000>
3 BOB measures S t:5
State:
1.0|0000>
Quantum system reset to: 0000
----- Case 1 -----
0 Initialize R t:1
State:
0.58|0000> + 0.82|0001>
1 ALICE observes R t:2
State:
0.58|0000> + 0.82|1001>
Projecting to subspace Alice_memory=1
1.0|1001>
2 Apply H to S controlled on ALICE_MEMORY t:4
State:
0.71|1001> + 0.71|1011>
3 BOB measures S t:5
State:
0.71|1001> + 0.71|1111>
Quantum system reset to: 0000


In:(Bob_memory:t5)    |  Out: (Alice_memory:t2)
------------------------------------------------------
           0          |        [0, 1]
           1          |        [1]

Ursula reasons about Bob, using backward inference as well.

In [20]:
p_TU_steps = [step1, step2, step3, step4, step5,
              step6, step7, step8 ,step9]
p_TU = sum(p_TU_steps)
p_TU

Step 0: Initialize R(t:1)
Step 1: ALICE observes R(t:2)
Step 2: ALICE makes an inference(t:3)
Step 3: Apply H to S controlled on ALICE_MEMORY(t:4)
Step 4: BOB measures S(t:5)
Step 5: BOB makes an inference(t:6)
Step 6: Reverse Alice reasoning (Step1: in ok --> 1(R)(t:7)
Step 7: Perform Hadamard on R (Step2: in ok --> 1(R)(t:8)
Step 8: URSULA measures ALICEs lab (i.e. r)(t:9)

Requirements: 
------------------------------
Qubit             ['r', 's']
AgentMemory(1)    ['Ursula']
Agent(1,1)        ['Bob', 'Alice']

In [21]:
TU = backward_inference(p_TU, 
                        subsys_x='Bob_memory', t_x=5, 
                        subsys_y='Ursula_memory', t_y=9, 
                        silent=False)
TU

(Note: This is the (slow) Python simulator.)
Require Qubit r
Require Qubit s
Require AgentMemory(1) Ursula
Require Agent(1,1) Bob
Require Agent(1,1) Alice
----- Case 0 -----
0 Initialize R t:1
State:
0.58|00000000000> + 0.82|00000000001>
1 ALICE observes R t:2
State:
0.58|00000000000> + 0.82|00010000001>
2 ALICE makes an inference t:3
State:
0.58|00000000000> + 0.82|00010000001>
3 Apply H to S controlled on ALICE_MEMORY t:4
State:
0.58|00000000000> + 0.58|00010000001> + 0.58|00010000011>
4 BOB measures S t:5
State:
0.58|00000000000> + 0.58|00010000001> + 0.58|00010001011>
Projecting to subspace Bob_memory=0
0.71|00000000000> + 0.71|00010000001>
5 BOB makes an inference t:6
State:
0.71|00000000000> + 0.71|00010000001>
6 Reverse Alice reasoning (Step1: in ok --> 1(R) t:7
State:
0.71|00000000000> + 0.71|00000000001>
7 Perform Hadamard on R (Step2: in ok --> 1(R) t:8
State:
1.0|00000000000>
8 URSULA measures ALICEs lab (i.e. r) t:9
State:
1.0|00000000000>
Quantum system reset to: 000000000

In:(Ursula_memory:t9) |  Out: (Bob_memory:t5)
----------------------------------------------------
           0          |        [0, 1]
           1          |        [1]

### Combining the inference tables with consistency

Now the consistency rules come to play. They tell us how to combine the obtained inference tables -- in this case we don't have any special restrictions, as we use the classical modal logic where we are always free to conclude $A \Rightarrow C$ from knowing $A \Rightarrow B$ and $B \Rightarrow C$, regardless of which agent has produced the statement.

In [22]:
from qthought.logicalReasoning.consistency import consistency

TA_final = TA
TB_final = consistency(TB, TA)
TU_final = consistency(TU, TB_final)

print(TA_final)
print(TB_final)
print(TU_final)

In:(Alice_memory:t2)  |  Out: (s:t13)
--------------------------------------------
           0          |        [0, 1]
           1          |        [0]
In:(Bob_memory:t5)    |  Out: (s:t13)
--------------------------------------------
           0          |        [0, 1]
           1          |        [0]
In:(Ursula_memory:t9) |  Out: (s:t13)
--------------------------------------------
           0          |        [0, 1]
           1          |        [0]


### Running the full protocol

Now we are ready to run the full protocol, and see if the "winning condition" (getting the inconsistency) is satisfied. You might need to run it several times as getting the inconsistency is probabilistic (we need to wait until getting to the round in which Ursula and Wigner both measure "ok").

In [23]:
steps = [step1, step2, step3, step4, step5,
         step6, step7, step8, step9, step10,
         step12, step13]
p = sum(steps)
p

Step 0: Initialize R(t:1)
Step 1: ALICE observes R(t:2)
Step 2: ALICE makes an inference(t:3)
Step 3: Apply H to S controlled on ALICE_MEMORY(t:4)
Step 4: BOB measures S(t:5)
Step 5: BOB makes an inference(t:6)
Step 6: Reverse Alice reasoning (Step1: in ok --> 1(R)(t:7)
Step 7: Perform Hadamard on R (Step2: in ok --> 1(R)(t:8)
Step 8: URSULA measures ALICEs lab (i.e. r)(t:9)
Step 9: URSULA makes inference(t:10)
Step 10: Reverse BOBs inference procedure(t:12)
Step 11: Apply Hadamard on S, i.e. transform system S+BOB:  ok --> 1(s) (t:13)

Requirements: 
------------------------------
Qubit             ['r', 's']
Agent(1,1)        ['Bob', 'Alice', 'Ursula']

In [25]:
print('-'*70)
print('Requiring quantum system:')
qsys = QuantumSystem(p.get_requirements())
no_prediction_state = 1
qsys.print_wavefunction()

print('-'*70)
print('Initialize inference system')
qsys['Alice'].set_inference_table(TA_final, no_prediction_state)
qsys['Bob'].set_inference_table(TB_final, no_prediction_state)
qsys['Ursula'].set_inference_table(TU_final, no_prediction_state)

qsys['Alice'].prep_inference()
qsys['Bob'].prep_inference()
qsys['Ursula'].prep_inference()
qsys.print_wavefunction()

print('-'*70)
print('Run protocol:')
p.run(qsys, silent=True)

print('-'*70)
print('Final Wavefunction before measurements:')
qsys.print_wavefunction()

print('-'*70)
print('Perform final measurements:')

print('Measure Ursulas meomry:')
Measure | qsys['Ursula_memory']
qsys.print_wavefunction()
ok_bar = bool(int(qsys.readout('Ursula_memory')))
#print('ok_bar=',ok_bar)

print('Measure Ursulas prediction:')
Measure | qsys['Ursula_prediction']
qsys.print_wavefunction()
#print(readout(qsys['Ursula_prediction']))
U_predicts_fail = bool(1 - int(qsys.readout('Ursula_prediction')))
#print('U_predicts_fail=',U_predicts_fail)

print('Measure s:')
Measure | qsys['s']
qsys.print_wavefunction()
ok = bool(int(qsys.readout('s')))
#print('ok=',ok)

print(qsys.subsystems[::-1])
del qsys

print('-'*70)
print('Analysis of winning condition:')
if ok_bar:
    print('Ursula observes ok_bar')
else:
    print('Ursula observes fail_bar' )


if U_predicts_fail:
    print('Ursula predicts fail')
else:
    print('Ursula does not know Alices prediction' )
    
if ok:
    print('Wigner observes ok')
else:
    print('Wigner observes fail' ) 
    
if ok and ok_bar:
    print('XXXXX Winning condition satisfied XXXXXX')

----------------------------------------------------------------------
Requiring quantum system:
(Note: This is the (slow) Python simulator.)
Require Qubit r
Require Qubit s
Require Agent(1,1) Bob
Require Agent(1,1) Alice
Require Agent(1,1) Ursula
1.0|00000000000000>
----------------------------------------------------------------------
Initialize inference system
1.0|01000100010000>
----------------------------------------------------------------------
Run protocol:
----------------------------------------------------------------------
Final Wavefunction before measurements:
-0.29|01010100010001> + 0.29|01010100010011> + 0.87|01100100010000> + 0.29|01100100010010>
----------------------------------------------------------------------
Perform final measurements:
Measure Ursulas meomry:
-0.71|01010100010001> + 0.71|01010100010011>
Measure Ursulas prediction:
-0.71|01010100010001> + 0.71|01010100010011>
Measure s:
1.0|01010100010011>
['Ursula', 'Alice', 'Bob', 's', 'r']
-----------------